# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,howrah,22.5892,88.3103,34.00,79,40,4.63,IN,1717939985
1,1,port-aux-francais,-49.3500,70.2167,6.76,98,100,17.41,TF,1717939971
2,2,tura,25.5198,90.2201,26.57,87,0,2.58,IN,1717939989
3,3,vorgashor,67.5833,63.9500,2.11,96,100,3.61,RU,1717939978
4,4,weno,7.4515,151.8468,26.97,100,100,0.00,FM,1717939987


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600    
)
# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [36]:
# Narrow down cities that fit criteria and drop any results with null values
idealcities = city_data_df[(city_data_df['Max Temp']<=5) & (city_data_df['Humidity']<=60)]
idealcities.head()

# Drop any rows with null values
idealcities= idealcities.dropna()

# Display sample data
idealcities


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
119,119,iqaluit,63.7506,-68.5145,2.61,51,100,2.57,CA,1717939984
167,167,longyearbyen,78.2186,15.6401,3.91,56,20,6.17,SJ,1717939974
276,276,susuman,62.7833,148.1667,3.88,58,100,0.89,RU,1717939985


### Step 3: Create a new DataFrame called `hotel_df`.

In [37]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealcities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
119,iqaluit,CA,63.7506,-68.5145,51,
167,longyearbyen,SJ,78.2186,15.6401,56,
276,susuman,RU,62.7833,148.1667,58,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [53]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "limit":1,
    "apiKey": geoapify_key,      
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=hotel_df.loc[index,"Lat"]
    longitude=hotel_df.loc[index,"Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
        
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?PARAMS"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

print(name_address)

Starting hotel search
iqaluit - nearest hotel: Frobisher Inn
longyearbyen - nearest hotel: The Vault
susuman - nearest hotel: No hotel found
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'country': 'Russia', 'country_code': 'ru', 'region': 'Far Eastern Federal District', 'state': 'Magadan Oblast', 'county': 'Susumansky Urban District', 'city': 'Susuman', 'postcode': '686310', 'street': 'улица Билибина', 'lon': 148.1511226, 'lat': 62.7767388, 'formatted': 'улица Билибина, Susuman, Magadan Oblast, Russia, 686310', 'address_line1': 'улица Билибина', 'address_line2': 'Susuman, Magadan Oblast, Russia, 686310', 'categories': ['accommodation', 'accommodation.hotel'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'osm_id': 644293626, 'tourism': 'hotel', 'osm_type': 'n'}}, 'distance': 1080, 'place_id': '51f815

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [58]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    hover_cols=["City","Country", "Hotel Name"],
    frame_width = 800,
    frame_height = 600    
)
# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)